# Fake Rejection:

Check efficiency plots w.r.t the reference (noringer)

In [1]:
from kepler.pandas.menu       import ElectronSequence as Chain
from kepler.pandas.readers    import load, load_in_loop, load_hdf
from core import Efficiency, restore_efficiencies
from Gaugi import mkdir_p, progressbar
from prettytable import PrettyTable
from tqdm import tqdm

import numpy as np
import pandas as pd
import collections
import os
import array
import root_numpy
import rootplotlib as rpl
from pprint import pprint
from copy import deepcopy
import gc
from ROOT import kBlack,kBlue,kRed,kAzure,kGreen,kMagenta,kCyan,kOrange,kGray,kYellow,kWhite,TColor,gStyle,TCanvas
import ROOT
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2
GeV = 1000.


Welcome to JupyROOT 6.16/00
Using all sub packages with ROOT dependence
INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.

Applying ATLAS style settings...

Applying ATLAS style settings...

Applying ATLAS style settings...


## 1) Read all files:

In [2]:
output_path = 'output/efficiencies'
mkdir_p(output_path)

In [3]:
# create my chain
chains = [
    Chain( "HLT_e17_lhvloose_nod0_noringer_L1EM15VHI", L1Seed = 'L1_EM15VHI' ),
    Chain( "HLT_e28_lhtight_nod0_noringer_ivarloose", L1Seed = 'L1_EM22VHI' ),
    Chain( "HLT_e60_lhmedium_nod0_noringer_L1EM24VHI", L1Seed = 'L1_EM24VHI' ),
    Chain( "HLT_e140_lhloose_nod0_noringer" , L1Seed = 'L1_EM24VHI' ),
    
    Chain( "HLT_e17_lhvloose_nod0_ringer_v17_L1EM15VHI"    , L1Seed = 'L1_EM15VHI' , l2calo_column = 'ringer_v13_vloose',l2_column = 'ringer_v3_el_loose'),
    Chain( "HLT_e28_lhtight_nod0_ringer_v17_ivarloose"     , L1Seed = 'L1_EM22VHI' , l2calo_column = 'ringer_v13_tight' ,l2_column = 'ringer_v3_el_loose'),
    Chain( "HLT_e60_lhmedium_nod0_ringer_v17_L1EM24VHI"    , L1Seed = 'L1_EM24VHI' , l2calo_column = 'ringer_v13_medium',l2_column = 'ringer_v3_el_loose'),
    Chain( "HLT_e140_lhloose_nod0_ringer_v17"              , L1Seed = 'L1_EM24VHI' , l2calo_column = 'ringer_v13_loose' ,l2_column = 'ringer_v3_el_loose'),
]

versions = ['v8', 'v9', 'v10', 'v11','v12','v13','v14','v15', 'v16']

for v in versions:
    
    chains.extend( [

            Chain( "HLT_e17_lhvloose_nod0_ringer_%s_L1EM15VHI"%v, L1Seed = 'L1_EM15VHI' , l2calo_column = 'ringer_%s_vloose'%v),
            Chain( "HLT_e28_lhtight_nod0_ringer_%s_ivarloose"%v, L1Seed = 'L1_EM22VHI' , l2calo_column = 'ringer_%s_tight'%v ),
            Chain( "HLT_e60_lhmedium_nod0_ringer_%s_L1EM24VHI"%v, L1Seed = 'L1_EM24VHI' , l2calo_column = 'ringer_%s_medium'%v),
            Chain( "HLT_e140_lhloose_nod0_ringer_%s"%v , L1Seed = 'L1_EM24VHI', l2calo_column = 'ringer_%s_loose'%v),

    ])


def emulate( df ):
    for chain in progressbar(chains, prefix='Emulate...'):
        chain.apply(df)

OrderedDict([('L1Seed', 'L1_EM15VHI'),
             ('L2Calo', (14000.0, 'trig_L2_cl_vloose_et12to22')),
             ('L2', 'trig_L2_el_cut_pt15to20'),
             ('EFCalo', 17000.0),
             ('HLT', (17000.0, 'trig_EF_el_lhvloose'))])
OrderedDict([('L1Seed', 'L1_EM22VHI'),
             ('L2Calo', (25000.0, 'trig_L2_cl_tight_et22toInf')),
             ('L2', 'trig_L2_el_cut_pt20to50'),
             ('EFCalo', 28000.0),
             ('HLT', (28000.0, 'trig_EF_el_lhtight_ivarloose'))])
OrderedDict([('L1Seed', 'L1_EM24VHI'),
             ('L2Calo', (57000.0, 'trig_L2_cl_medium_et22toInf')),
             ('L2', 'trig_L2_el_cut_pt50toInf'),
             ('EFCalo', 60000.0),
             ('HLT', (60000.0, 'trig_EF_el_lhmedium'))])
OrderedDict([('L1Seed', 'L1_EM24VHI'),
             ('L2Calo', (137000.0, 'trig_L2_cl_loose_et22toInf')),
             ('L2', 'trig_L2_el_cut_pt50toInf'),
             ('EFCalo', 140000.0),
             ('HLT', (140000.0, 'trig_EF_el_lhloose'))])
OrderedDic

# 2017:

In [4]:
dpath   = '../data/output/data17_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.dataframe.h5'

In [5]:
data_df = pd.concat( (load_hdf(dpath), 
                      load_hdf('../2021/v8/output/emulation/data17_table_v8.h5'),
                      load_hdf('v9/output/emulation/data17_table_v9.h5'),
                      load_hdf('v10/output/emulation/data17_table_v10.h5'),
                      load_hdf('v11/output/emulation/data17_table_v11.h5'),
                      load_hdf('../2021/v12/output/emulation/data17_table_v12.h5'),
                      load_hdf('v13/output/emulation/data17_table_v13.h5'),
                      load_hdf('v14/output/emulation/data17_table_v14.h5'),
                      load_hdf('v15/output/emulation/data17_table_v15.h5'),
                      load_hdf('v16/output/emulation/data17_table_v16.h5'),
                      load_hdf('v3_el/output/emulation/data17_table_v3_el.h5'),
                     ), axis=1)

In [6]:
emulate(data_df)

Emulate...: 100%|██████████| 44/44 [18:46<00:00, 25.61s/it] 


In [7]:
data_df.shape

(43311283, 378)

In [8]:
data_df = data_df.loc[ ((data_df['el_lhmedium']==True) & 
                    (data_df.target==1.0)) | ((data_df.target==0) & 
                    (data_df['el_lhvloose']==False) ) ]

data_df.shape

(39861985, 378)

In [9]:
triggers = [ 
            'e17_lhvloose_nod0_%s_L1EM15VHI', 
            'e28_lhtight_nod0_%s_ivarloose',
            'e60_lhmedium_nod0_%s_L1EM24VHI',
            'e140_lhloose_nod0_%s'
]
pidnames = [
            'el_lhvloose', 
            'el_lhtight', 
            'el_lhmedium',
            'el_lhloose'
]
et_cuts  = [17, 28, 60, 140]
versions = ['noringer',
            'ringer_v8', 
            'ringer_v9', 
            'ringer_v10', 
            'ringer_v11',
            'ringer_v12',
            'ringer_v13',
            'ringer_v14',
            'ringer_v15', 
            'ringer_v16',
            'ringer_v17']

for idx, trigger in enumerate(triggers):
    
    t = PrettyTable( ['version',
                      # FA
                      'L2CFA',
                      'L2CFA(<100)', 
                      'L2CFA(>100)',
                      'L2FA',
                      'L2FA(<100)',
                      'L2FA(>100)',
                      'HLT_FA', 
                      # PD
                      'L2CPD',
                      'L2PD' ,
                      'HLT_PD'] )
    
    is_sgn = (data_df.target==1)&(data_df[pidnames[idx]]==True)&(data_df['el_et']>(et_cuts[idx]+1)*GeV)
    is_bkg = (data_df.target==0)&(data_df['el_lhvloose']==False)&(data_df['el_et']>(et_cuts[idx]+1)*GeV)
    
    is_bkg_low = (data_df['el_et']<100*GeV)&is_bkg
    is_bkg_high= (data_df['el_et']>=100*GeV)&is_bkg
    is_sgn_low = (data_df['el_et']<100*GeV)&is_sgn
    is_sgn_high= (data_df['el_et']>=100*GeV)&is_sgn
    
    total_sgn  = len(data_df.loc[is_sgn])
    total_bkg  = len(data_df.loc[is_bkg])


    print(trigger%'X')
    for version in tqdm( versions ):
        
            values = [version]
            
            #
            # FA
            #
            
            # L2Calo FA
            passed = len(data_df.loc[is_bkg&(data_df['L2Calo_'+trigger%version]==True)])
            total  = len(data_df.loc[is_bkg])
            fa = passed/total * 100           
            values.append( '%1.2f'%(fa) )
            
            # L2Calo FA < 100
            total  = len(data_df.loc[is_bkg_low])
            if total > 0:
                passed = len(data_df.loc[is_bkg_low&(data_df['L2Calo_'+trigger%version]==True)])
                fa = passed/total * 100           
                values.append( '%1.2f'%(fa) )
            else:
                values.append( '%1.2f'%(0) )
               
            # L2Calo FA > 100
            passed = len(data_df.loc[is_bkg_high&(data_df['L2Calo_'+trigger%version]==True)])
            total  = len(data_df.loc[is_bkg_high])
            fa = passed/total * 100           
            values.append( '%1.2f'%(fa) )

            # L2 FA
            passed = len(data_df.loc[is_bkg&(data_df['L2_'+trigger%version]==True)])
            total  = len(data_df.loc[is_bkg])
            fa = passed/total * 100           
            values.append( '%1.2f'%(fa) )
            
            # L2 FA < 100
            total  = len(data_df.loc[is_bkg_low])
            if total > 0:
                passed = len(data_df.loc[is_bkg_low&(data_df['L2_'+trigger%version]==True)])
                fa = passed/total * 100           
                values.append( '%1.2f'%(fa) )
            else:
                values.append( '%1.2f'%(0) )
               
            # L2 FA > 100
            passed = len(data_df.loc[is_bkg_high&(data_df['L2_'+trigger%version]==True)])
            total  = len(data_df.loc[is_bkg_high])
            fa = passed/total * 100           
            values.append( '%1.2f'%(fa) )

            # HLT FA
            passed = len(data_df.loc[is_bkg&(data_df['HLT_'+trigger%version]==True)])
            fa = passed/total_bkg * 100           
            values.append( '%1.2f'%(fa) )
            
            
            #
            # PD
            #
            
            # L2Calo PD
            passed = len(data_df.loc[ is_sgn & (data_df['L2Calo_'+trigger%version]==True)])
            det = passed/total_sgn * 100
            values.append( '%1.2f'%(det) )
            
            
            # L2 PD
            passed = len(data_df.loc[ is_sgn & (data_df['L2_'+trigger%version]==True)])
            det = passed/total_sgn * 100
            values.append( '%1.2f'%(det) )
            
            
            # HLT PD
            passed = len(data_df.loc[ is_sgn & (data_df['HLT_'+trigger%version]==True)])
            det = passed/total_sgn * 100
            values.append( '%1.2f'%(det) )
            
            t.add_row(values)
    print(t)

            
            
            


e17_lhvloose_nod0_X_L1EM15VHI


100%|██████████| 11/11 [23:53<00:00, 130.32s/it]


+------------+-------+-------------+-------------+-------+------------+------------+--------+-------+-------+--------+
|  version   | L2CFA | L2CFA(<100) | L2CFA(>100) |  L2FA | L2FA(<100) | L2FA(>100) | HLT_FA | L2CPD |  L2PD | HLT_PD |
+------------+-------+-------------+-------------+-------+------------+------------+--------+-------+-------+--------+
|  noringer  | 35.83 |    13.46    |    43.98    | 35.77 |   13.44    |   43.90    |  0.29  | 94.48 | 94.34 | 92.17  |
| ringer_v8  |  2.24 |     2.58    |     2.12    |  2.16 |    2.56    |    2.02    |  0.17  | 94.66 | 94.51 | 92.34  |
| ringer_v9  |  1.92 |     1.90    |     1.92    |  1.84 |    1.88    |    1.82    |  0.14  | 94.67 | 94.52 | 92.36  |
| ringer_v10 |  5.25 |     2.19    |     6.36    |  4.65 |    2.17    |    5.55    |  0.15  | 94.66 | 94.51 | 92.34  |
| ringer_v11 |  3.11 |     1.69    |     3.62    |  2.73 |    1.67    |    3.11    |  0.13  | 94.65 | 94.51 | 92.34  |
| ringer_v12 |  2.14 |     2.55    |     1.98   

100%|██████████| 11/11 [19:33<00:00, 106.72s/it]


+------------+-------+-------------+-------------+-------+------------+------------+--------+-------+-------+--------+
|  version   | L2CFA | L2CFA(<100) | L2CFA(>100) |  L2FA | L2FA(<100) | L2FA(>100) | HLT_FA | L2CPD |  L2PD | HLT_PD |
+------------+-------+-------------+-------------+-------+------------+------------+--------+-------+-------+--------+
|  noringer  | 26.47 |     7.80    |    30.57    | 26.41 |    7.79    |   30.50    |  0.00  | 93.98 | 93.87 | 86.84  |
| ringer_v8  |  0.84 |     1.08    |     0.78    |  0.79 |    1.07    |    0.73    |  0.00  | 94.22 | 94.11 | 87.01  |
| ringer_v9  |  1.17 |     0.86    |     1.24    |  1.12 |    0.85    |    1.17    |  0.00  | 94.25 | 94.14 | 87.04  |
| ringer_v10 |  0.60 |     0.84    |     0.55    |  0.56 |    0.83    |    0.50    |  0.00  | 94.24 | 94.12 | 87.01  |
| ringer_v11 |  0.51 |     0.69    |     0.47    |  0.47 |    0.68    |    0.42    |  0.00  | 94.25 | 94.13 | 87.03  |
| ringer_v12 |  0.82 |     1.12    |     0.76   

100%|██████████| 11/11 [07:19<00:00, 39.93s/it]


+------------+-------+-------------+-------------+-------+------------+------------+--------+-------+-------+--------+
|  version   | L2CFA | L2CFA(<100) | L2CFA(>100) |  L2FA | L2FA(<100) | L2FA(>100) | HLT_FA | L2CPD |  L2PD | HLT_PD |
+------------+-------+-------------+-------------+-------+------------+------------+--------+-------+-------+--------+
|  noringer  | 34.45 |    17.60    |    35.03    | 34.37 |   17.59    |   34.95    |  0.01  | 97.31 | 97.22 | 91.87  |
| ringer_v8  |  0.83 |     0.38    |     0.85    |  0.78 |    0.37    |    0.80    |  0.00  | 97.36 | 97.27 | 91.90  |
| ringer_v9  |  1.57 |     0.50    |     1.61    |  1.49 |    0.50    |    1.53    |  0.00  | 97.37 | 97.28 | 91.90  |
| ringer_v10 |  0.67 |     0.42    |     0.68    |  0.61 |    0.42    |    0.62    |  0.00  | 97.21 | 97.12 | 91.76  |
| ringer_v11 |  0.55 |     0.33    |     0.55    |  0.49 |    0.33    |    0.50    |  0.00  | 97.15 | 97.06 | 91.71  |
| ringer_v12 |  0.81 |     0.44    |     0.83   

100%|██████████| 11/11 [05:04<00:00, 27.69s/it]

+------------+-------+-------------+-------------+-------+------------+------------+--------+-------+-------+--------+
|  version   | L2CFA | L2CFA(<100) | L2CFA(>100) |  L2FA | L2FA(<100) | L2FA(>100) | HLT_FA | L2CPD |  L2PD | HLT_PD |
+------------+-------+-------------+-------------+-------+------------+------------+--------+-------+-------+--------+
|  noringer  | 94.57 |     0.00    |    94.57    | 69.06 |    0.00    |   69.06    |  0.07  | 98.19 | 98.13 | 95.15  |
| ringer_v8  |  2.06 |     0.00    |     2.06    |  1.96 |    0.00    |    1.96    |  0.05  | 98.16 | 98.09 | 95.14  |
| ringer_v9  |  2.14 |     0.00    |     2.14    |  2.02 |    0.00    |    2.02    |  0.04  | 98.20 | 98.14 | 95.19  |
| ringer_v10 |  4.28 |     0.00    |     4.28    |  3.83 |    0.00    |    3.83    |  0.05  | 98.14 | 98.08 | 95.12  |
| ringer_v11 |  2.56 |     0.00    |     2.56    |  2.26 |    0.00    |    2.26    |  0.04  | 98.09 | 98.02 | 95.07  |
| ringer_v12 |  1.92 |     0.00    |     1.92   

# 2018:

In [10]:
dpath   = '../data/output/data18_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.dataframe.h5'

In [11]:
data_df = pd.concat( (load_hdf(dpath), 
                      load_hdf('../2021/v8/output/emulation/data18_table_v8.h5'),
                      load_hdf('v9/output/emulation/data18_table_v9.h5'),
                      load_hdf('v10/output/emulation/data18_table_v10.h5'),
                      load_hdf('v11/output/emulation/data18_table_v11.h5'),
                      load_hdf('v12/output/emulation/data18_table_v12.h5'),
                      load_hdf('v13/output/emulation/data18_table_v13.h5'),
                      load_hdf('v14/output/emulation/data18_table_v14.h5'),
                      load_hdf('v15/output/emulation/data18_table_v15.h5'),
                      load_hdf('v16/output/emulation/data18_table_v16.h5'),
                      load_hdf('v3_el/output/emulation/data18_table_v3_el.h5'),
                     ), axis=1)

In [12]:
data_df = data_df.loc[ ((data_df['el_lhmedium']==True) & 
                    (data_df.target==1.0)) | ((data_df.target==0) & 
                    (data_df['el_lhvloose']==False) ) ]

data_df.shape

(38543015, 158)

In [13]:
emulate(data_df)

Emulate...: 100%|██████████| 44/44 [16:39<00:00, 22.72s/it]


In [14]:
triggers = [ 
            'e17_lhvloose_nod0_%s_L1EM15VHI', 
            'e28_lhtight_nod0_%s_ivarloose',
            'e60_lhmedium_nod0_%s_L1EM24VHI',
            'e140_lhloose_nod0_%s'
]
pidnames = [
            'el_lhvloose', 
            'el_lhtight', 
            'el_lhmedium',
            'el_lhloose'
]
et_cuts  = [17, 28, 60, 140]
versions = ['noringer',
            'ringer_v8', 
            'ringer_v9', 
            'ringer_v10', 
            'ringer_v11',
            'ringer_v12',
            'ringer_v13',
            'ringer_v14',
            'ringer_v15', 
            'ringer_v16',
            'ringer_v17']

for idx, trigger in enumerate(triggers):
    
    t = PrettyTable( ['version',
                      # FA
                      'L2CFA',
                      'L2CFA(<100)', 
                      'L2CFA(>100)',
                      'L2FA',
                      'L2FA(<100)',
                      'L2FA(>100)',
                      'HLT_FA', 
                      # PD
                      'L2CPD',
                      'L2PD' ,
                      'HLT_PD'] )
    
    is_sgn = (data_df.target==1)&(data_df[pidnames[idx]]==True)&(data_df['el_et']>(et_cuts[idx]+1)*GeV)
    is_bkg = (data_df.target==0)&(data_df['el_lhvloose']==False)&(data_df['el_et']>(et_cuts[idx]+1)*GeV)
    
    is_bkg_low = (data_df['el_et']<100*GeV)&is_bkg
    is_bkg_high= (data_df['el_et']>=100*GeV)&is_bkg
    is_sgn_low = (data_df['el_et']<100*GeV)&is_sgn
    is_sgn_high= (data_df['el_et']>=100*GeV)&is_sgn
    
    total_sgn  = len(data_df.loc[is_sgn])
    total_bkg  = len(data_df.loc[is_bkg])


    print(trigger%'X')
    for version in tqdm( versions ):
        
            values = [version]
            
            #
            # FA
            #
            
            # L2Calo FA
            passed = len(data_df.loc[is_bkg&(data_df['L2Calo_'+trigger%version]==True)])
            total  = len(data_df.loc[is_bkg])
            fa = passed/total * 100           
            values.append( '%1.2f'%(fa) )
            
            # L2Calo FA < 100
            total  = len(data_df.loc[is_bkg_low])
            if total > 0:
                passed = len(data_df.loc[is_bkg_low&(data_df['L2Calo_'+trigger%version]==True)])
                fa = passed/total * 100           
                values.append( '%1.2f'%(fa) )
            else:
                values.append( '%1.2f'%(0) )
               
            # L2Calo FA > 100
            passed = len(data_df.loc[is_bkg_high&(data_df['L2Calo_'+trigger%version]==True)])
            total  = len(data_df.loc[is_bkg_high])
            fa = passed/total * 100           
            values.append( '%1.2f'%(fa) )

            # L2 FA
            passed = len(data_df.loc[is_bkg&(data_df['L2_'+trigger%version]==True)])
            total  = len(data_df.loc[is_bkg])
            fa = passed/total * 100           
            values.append( '%1.2f'%(fa) )
            
            # L2 FA < 100
            total  = len(data_df.loc[is_bkg_low])
            if total > 0:
                passed = len(data_df.loc[is_bkg_low&(data_df['L2_'+trigger%version]==True)])
                fa = passed/total * 100           
                values.append( '%1.2f'%(fa) )
            else:
                values.append( '%1.2f'%(0) )
               
            # L2 FA > 100
            passed = len(data_df.loc[is_bkg_high&(data_df['L2_'+trigger%version]==True)])
            total  = len(data_df.loc[is_bkg_high])
            fa = passed/total * 100           
            values.append( '%1.2f'%(fa) )

            # HLT FA
            passed = len(data_df.loc[is_bkg&(data_df['HLT_'+trigger%version]==True)])
            fa = passed/total_bkg * 100           
            values.append( '%1.2f'%(fa) )
            
            
            #
            # PD
            #
            
            # L2Calo PD
            passed = len(data_df.loc[ is_sgn & (data_df['L2Calo_'+trigger%version]==True)])
            det = passed/total_sgn * 100
            values.append( '%1.2f'%(det) )
            
            
            # L2 PD
            passed = len(data_df.loc[ is_sgn & (data_df['L2_'+trigger%version]==True)])
            det = passed/total_sgn * 100
            values.append( '%1.2f'%(det) )
            
            
            # HLT PD
            passed = len(data_df.loc[ is_sgn & (data_df['HLT_'+trigger%version]==True)])
            det = passed/total_sgn * 100
            values.append( '%1.2f'%(det) )
            
            t.add_row(values)
    print(t)

            
            

e17_lhvloose_nod0_X_L1EM15VHI


100%|██████████| 11/11 [24:29<00:00, 133.60s/it]


+------------+-------+-------------+-------------+-------+------------+------------+--------+-------+-------+--------+
|  version   | L2CFA | L2CFA(<100) | L2CFA(>100) |  L2FA | L2FA(<100) | L2FA(>100) | HLT_FA | L2CPD |  L2PD | HLT_PD |
+------------+-------+-------------+-------------+-------+------------+------------+--------+-------+-------+--------+
|  noringer  | 36.80 |    13.77    |    44.41    | 36.74 |   13.76    |   44.34    |  1.02  | 94.45 | 94.32 | 92.34  |
| ringer_v8  |  2.71 |     3.03    |     2.61    |  2.63 |    3.01    |    2.50    |  0.69  | 94.64 | 94.50 | 92.52  |
| ringer_v9  |  2.29 |     2.37    |     2.27    |  2.22 |    2.36    |    2.17    |  0.63  | 94.64 | 94.51 | 92.53  |
| ringer_v10 |  5.71 |     2.72    |     6.70    |  5.01 |    2.70    |    5.78    |  0.63  | 94.63 | 94.50 | 92.52  |
| ringer_v11 |  3.59 |     2.22    |     4.04    |  3.14 |    2.21    |    3.45    |  0.58  | 94.62 | 94.49 | 92.51  |
| ringer_v12 |  2.72 |     2.92    |     2.65   

100%|██████████| 11/11 [18:59<00:00, 103.59s/it]


+------------+-------+-------------+-------------+-------+------------+------------+--------+-------+-------+--------+
|  version   | L2CFA | L2CFA(<100) | L2CFA(>100) |  L2FA | L2FA(<100) | L2FA(>100) | HLT_FA | L2CPD |  L2PD | HLT_PD |
+------------+-------+-------------+-------------+-------+------------+------------+--------+-------+-------+--------+
|  noringer  | 27.30 |     8.31    |    31.07    | 27.24 |    8.30    |   31.00    |  0.06  | 93.93 | 93.84 | 87.38  |
| ringer_v8  |  1.20 |     1.54    |     1.13    |  1.15 |    1.53    |    1.08    |  0.05  | 94.19 | 94.09 | 87.58  |
| ringer_v9  |  1.53 |     1.33    |     1.57    |  1.48 |    1.33    |    1.51    |  0.05  | 94.22 | 94.12 | 87.61  |
| ringer_v10 |  0.92 |     1.38    |     0.82    |  0.87 |    1.38    |    0.77    |  0.04  | 94.21 | 94.11 | 87.59  |
| ringer_v11 |  0.82 |     1.25    |     0.74    |  0.78 |    1.24    |    0.69    |  0.04  | 94.21 | 94.11 | 87.59  |
| ringer_v12 |  1.82 |     1.53    |     1.88   

100%|██████████| 11/11 [06:20<00:00, 34.58s/it]


+------------+-------+-------------+-------------+-------+------------+------------+--------+-------+-------+--------+
|  version   | L2CFA | L2CFA(<100) | L2CFA(>100) |  L2FA | L2FA(<100) | L2FA(>100) | HLT_FA | L2CPD |  L2PD | HLT_PD |
+------------+-------+-------------+-------------+-------+------------+------------+--------+-------+-------+--------+
|  noringer  | 34.98 |    17.88    |    35.51    | 34.91 |   17.88    |   35.43    |  0.08  | 97.31 | 97.22 | 92.14  |
| ringer_v8  |  1.19 |     0.58    |     1.21    |  1.14 |    0.57    |    1.16    |  0.06  | 97.35 | 97.26 | 92.19  |
| ringer_v9  |  1.95 |     0.69    |     1.99    |  1.88 |    0.69    |    1.91    |  0.07  | 97.35 | 97.26 | 92.19  |
| ringer_v10 |  0.96 |     0.62    |     0.98    |  0.91 |    0.62    |    0.92    |  0.05  | 97.23 | 97.14 | 92.08  |
| ringer_v11 |  0.83 |     0.54    |     0.84    |  0.78 |    0.54    |    0.78    |  0.05  | 97.16 | 97.07 | 92.02  |
| ringer_v12 |  2.01 |     0.74    |     2.05   

100%|██████████| 11/11 [04:22<00:00, 23.85s/it]

+------------+-------+-------------+-------------+-------+------------+------------+--------+-------+-------+--------+
|  version   | L2CFA | L2CFA(<100) | L2CFA(>100) |  L2FA | L2FA(<100) | L2FA(>100) | HLT_FA | L2CPD |  L2PD | HLT_PD |
+------------+-------+-------------+-------------+-------+------------+------------+--------+-------+-------+--------+
|  noringer  | 94.64 |     0.00    |    94.64    | 64.74 |    0.00    |   64.74    |  0.43  | 98.20 | 98.13 | 95.45  |
| ringer_v8  |  2.62 |     0.00    |     2.62    |  2.52 |    0.00    |    2.52    |  0.36  | 98.23 | 98.16 | 95.49  |
| ringer_v9  |  2.55 |     0.00    |     2.55    |  2.44 |    0.00    |    2.44    |  0.35  | 98.27 | 98.20 | 95.53  |
| ringer_v10 |  4.75 |     0.00    |     4.75    |  4.24 |    0.00    |    4.24    |  0.32  | 98.13 | 98.06 | 95.40  |
| ringer_v11 |  3.04 |     0.00    |     3.04    |  2.70 |    0.00    |    2.70    |  0.31  | 98.04 | 97.97 | 95.33  |
| ringer_v12 |  2.97 |     0.00    |     2.97   